In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Generate data
np.random.seed(42)
X = np.linspace(-5, 5, 100).reshape(-1, 1)
y_true = 2 * X.flatten() + 3  # Linear function: y = 2x + 3
y = y_true + np.random.normal(0, 2, size=X.shape[0])  # Add Gaussian noise

# Split into training and testing sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to fit and evaluate models
def fit_and_evaluate(degree):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Degree {degree} - MSE: {mse:.4f}")
    return model

# Fit models with different polynomial degrees
degrees = [1, 2, 3]
models = {d: fit_and_evaluate(d) for d in degrees}

# Plot results
plt.scatter(X, y, color='gray', label='Data', alpha=0.5)
x_range = np.linspace(-5, 5, 100).reshape(-1, 1)
colors = ['r', 'g', 'b']
labels = ['Linear', 'Quadratic', 'Cubic']

for i, d in enumerate(degrees):
    y_range_pred = models[d].predict(x_range)
    plt.plot(x_range, y_range_pred, colors[i], label=f'{labels[i]} Fit')

plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.title("Polynomial Regression Fits")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
file_path = "p2.csv"  # Update with the correct path if needed
df = pd.read_csv(file_path)

# Extract features and target variable
X = df[['Feature 1', 'Feature 2']].values  # Features
y = df['Output'].values  # Target

# Normalize features for better convergence
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Add bias term (intercept) to X
X = np.c_[np.ones(X.shape[0]), X]

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Compute Mean Squared Error
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Logistic Regression using Gradient Descent
def logistic_regression(X, y, lr, iterations):
    m, n = X.shape
    weights = np.zeros(n)  # Initialize weights
    mse_values = []

    for _ in range(iterations):
        z = np.dot(X, weights)
        y_pred = sigmoid(z)

        # Compute gradient
        gradient = np.dot(X.T, (y_pred - y)) / m

        # Update weights
        weights -= lr * gradient

        # Compute MSE for monitoring
        mse_values.append(mse(y, y_pred))

    return weights, mse_values

# Train the model for both learning rates
iterations = 20
learning_rates = [0.01, 0.05]
results = {}

for lr in learning_rates:
    weights, mse_values = logistic_regression(X, y, lr, iterations)
    results[lr] = (weights, mse_values)
    print(f"Learning Rate: {lr}\nFinal Weights: {weights}\n")

# Plot MSE variation for both learning rates
plt.figure(figsize=(10, 5))
for lr in learning_rates:
    plt.plot(range(1, iterations + 1), results[lr][1], label=f'LR = {lr}')

plt.xlabel('Iterations')
plt.ylabel('Mean Squared Error')
plt.title('MSE Variation Over Iterations')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

# Load dataset
data = pd.read_csv('p3.csv')

# Assuming the last column is the target variable
y = data.iloc[:, -1]
X = data.iloc[:, :-1]

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
lr = LinearRegression()
lasso = Lasso(alpha=2)  # lambda/2 = 1, so alpha = 2
ridge = Ridge(alpha=0.2)  # lambda/2 = 0.1, so alpha = 0.2

# Train models
lr.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)

# Predict on test set
y_pred_lr = lr.predict(X_test)
y_pred_lasso = lasso.predict(X_test)
y_pred_ridge = ridge.predict(X_test)

# Compute MSE
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

# Bar plot for MSE comparison
plt.figure(figsize=(8, 5))
plt.bar(['Linear', 'LASSO', 'Ridge'], [mse_lr, mse_lasso, mse_ridge], color=['blue', 'red', 'green'])
plt.ylabel('MSE')
plt.title('MSE Comparison')
plt.show()

# Bar plot for feature coefficients
coeffs = pd.DataFrame({'Feature': X.columns,
                        'Linear': lr.coef_,
                        'LASSO': lasso.coef_,
                        'Ridge': ridge.coef_})

coeffs.set_index('Feature').plot(kind='bar', figsize=(10, 6))
plt.ylabel('Coefficient Value')
plt.title('Feature Coefficients Comparison')
plt.show()

# Print MSE values
print(f"Linear Regression MSE: {mse_lr}")
print(f"LASSO Regression MSE: {mse_lasso}")
print(f"Ridge Regression MSE: {mse_ridge}")
